# Convert data from the M2HATS field campaign to Zarr

This notebook utilizes the GDEX to read in, standardize, and convert files relevant to the M2HATS field campaign to Zarr for more efficient data processing. The two datasets used in this example are ERA5 (stored on GDEX) and 30-minute 449MHz Wind Profiler data (from the EOL FDA and temporarily stored on GDEX), but it is a straightforward process that could be applied to all ISS instruments during any field campaign. 

### Imports

In [1]:
# For analysis code
import glob
import numpy as np
import xarray as xr
import pandas as pd
import metpy.calc as mpcalc

# For Dask + cluster
from dask_jobqueue import PBSCluster
from distributed import Client
from dask import delayed

### Scratch directory
Designated scratch directory to hold Zarr stores created from field campaign data + ERA5 model data stored as NetCDF files.

In [2]:
lustre_scratch  = "/lustre/desc1/scratch/myasears"

### Spin up a cluster

In [3]:
cluster = PBSCluster(
        job_name = 'dask-eol-25',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = lustre_scratch + '/dask/spill',
        log_directory = lustre_scratch + '/dask/logs/',
        resource_spec = 'select=1:ncpus=1:mem=4GB',
        queue = 'casper',
        walltime = '3:00:00',
        interface = 'ext')

/glade/u/apps/opt/conda/envs/npl-2025b/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40195 instead
  warnings.warn(


In [4]:
client = Client(cluster)

In [5]:
n_workers = 5
cluster.scale(n_workers)
client.wait_for_workers(n_workers = n_workers)

### Load ERA5 data
ERA5 reanalysis information is stored on GDEX, so we can pull it right from there. However, we are interested in atmospheric profiles form a single lat/lon point for this analysis, and the data are stored on pressure levels over an xy plane spanning the entire globe. To work with this information, we lazily load the all relevant monthly datasets for a single variable, and subset the Xarray Dataset by the lat/lon of the profiler and all times spanning the target field campaign. This process is repeated for all desired variables, then all resulting datasets are merged together to produce an all-inclusive dataset for the field campaign. 

In [ ]:
era5_path = '/gdex/data/d633000/e5.oper.an.pl'

In [ ]:
target_lat = 38.0
target_lon = 243.0

start_date = pd.Timestamp("2023-07-11T00:00:00")
end_date = pd.Timestamp("2023-09-27T23:59:59")
yyyymm = ["202307", "202308", "202309"]

var_map = {"Z": "e5.oper.an.pl.128_129_z",
           "U": "e5.oper.an.pl.128_131_u",
           "V": "e5.oper.an.pl.128_132_v",
           "W": "e5.oper.an.pl.128_135_w"
           }

In [ ]:
def open_variable(file_prefix, yyyymm):
    files = []
    for month in yyyymm:
        files.extend(sorted(glob.glob(f'{era5_path}/{month}/{file_prefix}*')))

    ds = xr.open_mfdataset(files, combine="by_coords", parallel=True)
    ds_point = ds.sel(latitude=target_lat, longitude=target_lon, time=slice(start_date, end_date))
    
    return ds_point

In [ ]:
# Open and subset all variables
datasets = [open_variable(file_prefix, yyyymm) for file_prefix in var_map.values()]

# Merge them together
combined_era5 = xr.merge(datasets, compat="override", combine_attrs="override")

In [ ]:
# Convert geopotential to geometric height (m above MSL)
height = (combined_era5["Z"] * 6371008.7714) / (9.80665 * 6371008.7714 - combined_era5["Z"])
combined_era5["height_msl"] = height
combined_era5.height_msl.attrs.update({"long_name": "Height above mean sea level", "units": "meters"})

# Drop utc_date variable
combined_era5 = combined_era5.drop_vars("utc_date")

# Change variable names to standardize with other datasets
name_mapping = {"level": "pressure", "Z": "geopotential", "U": "u_wind", "V": "v_wind", "W": "w_wind"}
combined_era5 = combined_era5.rename(name_mapping)

In [ ]:
combined_era5.to_zarr(f"{lustre_scratch}/2023_M2HATS/era5_M2HATS_ISS2.zarr")

### Load 449 data

In [26]:
prof449_path = "/gdex/data/special_projects/pythia_2025/eol-cookbook/m2hats_iss2_data/prof449Mhz_30min_winds"

In [27]:
files = []
files.extend(sorted(glob.glob(f'{prof449_path}/*.nc')))

In [28]:
# Get min and max altitude from campaign 

def get_minmax_alt(f):
    with xr.open_dataset(f, decode_cf=False) as tmp:
        return float(tmp['height'].min()), float(tmp['height'].max())

min_heights, max_heights = zip(*[get_minmax_alt(f) for f in files])
min_height, max_height = min(min_heights), max(max_heights)

In [29]:
# Retrieve common height grid (with a step of 100m) using max and min values
step = 100
# Create common height grid
common_agl = np.arange(min_height, max_height + step, step)

# Retrieve altitude value from fifth file (after setup -- checked manually)
altitude = xr.open_dataset(files[5]).alt.values

# Use alt to create common MSL grid
common_msl = common_agl + altitude

In [30]:
def open_and_regrid(f, common_agl, common_msl):
    ds = xr.open_dataset(f, chunks="auto")

    # Calculate MSL height from AGL
    msl_height = ds['height'].isel(time=0) + altitude

    # Make height coordinate 1-dimensional (same at every time step)
    height_1d = ds['height'].isel(time=0).values
    ds = ds.assign_coords(height=("height", height_1d))

    # Reindex height coords to span min + max from entire campaign
    ds = ds.reindex(height=common_agl)
    
    # Update coords to the reindexed grid
    ds = ds.assign_coords(
        height_agl=("height", common_agl),
        height_msl=("height", common_msl)
    )
    
    ds.height_msl.attrs.update({"long_name": "Height above mean sea level", "units": "meters"})
    
    # Swap to make geopotential the vertical coordinate
    ds = ds.swap_dims({"height": "height_msl"}).drop_vars("height")

    return ds

In [31]:
datasets = [delayed(open_and_regrid)(f, common_agl, common_msl) for f in files[2:]]
datasets = [d.compute() for d in datasets]
combined_profiler = xr.concat(datasets, dim="time", combine_attrs="override")

In [32]:
combined_profiler = combined_profiler.assign_coords(
    latitude=combined_profiler["lat"].isel(time=0).item(),
    longitude=combined_profiler["lon"].isel(time=0).item(),
    altitude=combined_profiler["alt"].isel(time=0).item()
).drop_vars(["lat", "lon", "alt"])

name_mapping = {
    "u": "u_wind",
    "v": "v_wind",
    "wvert": "w_wind"
}

vars_to_keep = [var for var in name_mapping if var in combined_profiler.data_vars]
combined_profiler = combined_profiler.rename(name_mapping)

In [34]:
combined_profiler = combined_profiler.chunk({"time": 48, "height_msl": -1})
combined_profiler.to_zarr(f"{lustre_scratch}/2023_M2HATS/prof449_M2HATS_ISS1_winds30.zarr")

/glade/derecho/scratch/myasears/tmp/ipykernel_9465/2820316678.py:2: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  combined_profiler.to_zarr(f"{lustre_scratch}/2023_M2HATS/prof449_M2HATS_ISS1_winds30.zarr")
/glade/u/apps/opt/conda/envs/npl-2025b/lib/python3.12/site-packages/zarr/api/asynchronous.py:229: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


### Open the Zarr files for confirmation

In [36]:
era5_test_zarr = xr.open_zarr(f"{lustre_scratch}/2023_M2HATS/era5_M2HATS_ISS2.zarr")
era5_test_zarr

<xarray.Dataset> Size: 1MB
Dimensions:       (time: 1896, pressure: 37)
Coordinates:
  * pressure      (pressure) float64 296B 1.0 2.0 3.0 5.0 ... 950.0 975.0 1e+03
    latitude      float64 8B ...
    longitude     float64 8B ...
  * time          (time) datetime64[ns] 15kB 2023-07-11 ... 2023-09-27T23:00:00
Data variables:
    height_msl    (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    geopotential  (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    w_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    v_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
    u_wind        (time, pressure) float32 281kB dask.array<chunksize=(1, 37), meta=np.ndarray>
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.8.1
    CONVERSION_PLATFORM:  Linux r2i4n28 4.12.14-95.51-default #1 SMP Fri Apr ...
    CONVERSION_DATE:      Fri Oct  6 09:19:55 MDT 2023
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Fri Oct  6 09:20:11 2023: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 5.0.3 (Homepage = http://n...

In [35]:
prof449Mhz_test_zarr = xr.open_zarr(f"{lustre_scratch}/2023_M2HATS/prof449_M2HATS_ISS1_winds30.zarr")
prof449Mhz_test_zarr

<xarray.Dataset> Size: 15MB
Dimensions:       (time: 3696, height_msl: 97)
Coordinates:
    latitude      float64 8B ...
    longitude     float64 8B ...
    height_agl    (height_msl) float64 776B dask.array<chunksize=(97,), meta=np.ndarray>
  * height_msl    (height_msl) float64 776B 1.841e+03 1.941e+03 ... 1.144e+04
    altitude      float64 8B ...
  * time          (time) datetime64[ns] 30kB 2023-07-13T00:15:00 ... 2023-09-...
Data variables: (12/15)
    wdir          (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
    v_dispersion  (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
    time_offset   (time) datetime64[ns] 30kB dask.array<chunksize=(48,), meta=np.ndarray>
    v_wind        (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
    wspd          (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
    x_spacing     (time) float32 15kB dask.array<chunksize=(48,), meta=np.ndarray>
    ...            ...
    u_dispersion  (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
    base_time     (time) datetime64[ns] 30kB dask.array<chunksize=(48,), meta=np.ndarray>
    specWid       (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
    cons_npoints  (time, height_msl) int16 717kB dask.array<chunksize=(48, 97), meta=np.ndarray>
    w_wind        (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
    u_wind        (time, height_msl) float32 1MB dask.array<chunksize=(48, 97), meta=np.ndarray>
Attributes: (12/16)
    description:        Modular Profiler consensus winds
    Processing-Method:  30 minute consensus averages
    Processing-Ops:       read_raw_winds mapr_nc szl yyyy correct_szl correct...
    history:            write_cons_5.pro
    author:             Bill Brown, NCAR/EOL
    date:               Mon Jul  1 09:54:45 2024   1719849285
    ...                 ...
    project:            M2HATS
    site:               Tonopah Airport, NV
    missing_value:      -999.0
    zebra_platform:     MAPR/449 winds
    first_file:         /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...
    last_file:          /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...